In [39]:
import pandas as pd

In [40]:
df_seoul = pd.read_csv('../data/bike_clean.csv')
df_seoul.head()

,Unnamed: 0,datetime,date,hour,season,holiday,open,rent_count,temperature,humidity,wind_speed,visibility,dewpoint_temp,solar_radiation,rainfall,snowfall
0,1,2017-12-01 00:00:00,2017-12-01,0,Winter,No Holiday,Yes,254,-5.2,37,2.2,20000,-17.6,0.0,0.0,0
1,2,2017-12-01 01:00:00,2017-12-01,1,Winter,No Holiday,Yes,204,-5.5,38,0.8,20000,-17.6,0.0,0.0,0
2,3,2017-12-01 02:00:00,2017-12-01,2,Winter,No Holiday,Yes,173,-6.0,39,1.0,20000,-17.7,0.0,0.0,0
3,4,2017-12-01 03:00:00,2017-12-01,3,Winter,No Holiday,Yes,107,-6.2,40,0.9,20000,-17.6,0.0,0.0,0
4,5,2017-12-01 04:00:00,2017-12-01,4,Winter,No Holiday,Yes,78,-6.0,36,2.3,20000,-18.6,0.0,0.0,0


In [41]:
df_seoul.dtypes

Unnamed: 0           int64
datetime            object
date                object
hour                 int64
season              object
holiday             object
open                object
rent_count           int64
temperature        float64
humidity             int64
wind_speed         float64
visibility           int64
dewpoint_temp      float64
solar_radiation    float64
rainfall           float64
snowfall             int64
dtype: object

### Datetime Feature Engineering

In [47]:
df_seoul['datetime'] =  pd.to_datetime(df_seoul['datetime'])
df_seoul['month'] = df_seoul.datetime.dt.month
df_seoul['week'] = df_seoul.datetime.dt.isocalendar().week # week number
df_seoul['day'] = df_seoul.datetime.dt.day # 1 to 31
df_seoul['dayofyear'] = df_seoul.datetime.dt.dayofyear # 1 to 365
df_seoul['dayofweek'] = df_seoul.datetime.dt.dayofweek # mon, tue ... 

In [43]:
df_seoul.month.value_counts(dropna=False)

12    744
1     744
3     744
5     744
7     744
8     744
10    744
4     720
6     720
9     720
11    720
2     672
Name: month, dtype: int64

In [44]:
df_seoul.week.value_counts(dropna=False)

48     192
21     168
50     168
51     168
52     168
1      168
2      168
3      168
4      168
5      168
6      168
7      168
9      168
20     168
10     168
11     168
12     168
13     168
14     168
15     168
16     168
17     168
18     168
19     168
47     168
8      168
22     168
49     168
23     168
24     168
25     168
26     168
27     168
28     168
29     168
30     168
31     168
32     168
33     168
34     168
35     168
36     168
37     168
38     168
39     168
40     168
41     168
42     168
43     168
44     168
45     168
46     168
NaN      0
Name: week, dtype: Int64

In [45]:
df_seoul.day.value_counts(dropna=False)

1     288
2     288
28    288
27    288
26    288
25    288
24    288
23    288
22    288
21    288
20    288
19    288
18    288
17    288
16    288
15    288
14    288
13    288
12    288
11    288
10    288
9     288
8     288
7     288
6     288
5     288
4     288
3     288
29    264
30    264
31    168
Name: day, dtype: int64

In [46]:
df_seoul.dayofweek.value_counts(dropna=False)
# should we change this to string?

4    1272
5    1248
6    1248
0    1248
1    1248
2    1248
3    1248
Name: dayofweek, dtype: int64

### Automated Feature Engineering